In [1]:
import tensorflow as tf
# tf.test.gpu_device_name()
tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)

True

In [2]:
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
print('sess: ', sess)

sess:  <tensorflow.python.client.session.Session object at 0x7f33b8cc14a8>


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1658153053143012380
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 5833999399566463469
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 5612226178552315962
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11329617920
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17727615171892673228
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
import notebook
print(notebook.__version__)

5.0.0


In [1]:
import model_2_branch
import common_function
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.optimizers import SGD, Adam
import os
import asyncio
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.models import load_model, Model

# Just disables the warning, doesn't enable AVX/FMA (no GPU)
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

epochs = 40
l_rate = 1.0e-4
decay = l_rate / epochs
sgd = Adam(lr=l_rate, beta_1=0.9, beta_2=0.999, amsgrad=False)
batch_size = 32
img_width, img_height = 24, 24
path_data_set = './ytd'
input_img, merged = model_2_branch.get_model(img_width, img_height)
num_train_images = 424961  # training images: 424961  # total images: 605855
file_path = 'tbe_cnn_ytd_2_branch.h5'

datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)


############################################### Training Dataset #############################################################


if not os.path.exists(file_path):
    flatten = Flatten()(merged)
    dense = Dense(64)(flatten)
    activation = Activation('softmax')(dense)
    dropout = Dropout(0.5)(activation)
    dense = Dense(1591)(dropout)
    activation = Activation('softmax')(dense)

    base_model = Model(input_img, activation)
else:
    base_model = load_model(file_path)
    for layer in base_model.layers:
        if layer.name is 'inception4_branch_3' \
                and layer.name is 'inception4d_activation_branch_3' \
                and layer.name is 'inception4e_activation_branch_3' \
                and layer.name is 'inception5a_activation_branch_3' \
                and layer.name is 'inception5b_4_branch_3':
            layer.trainable = False
    # base_model.load_weights(file_path)

base_model.summary()

train_generator_lr = datagen.flow_from_directory(
    str(path_data_set + '/train'),
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

validation_generator_lr = datagen.flow_from_directory(
    str(path_data_set + '/validate'),
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

print('training: ')

base_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

checkpoint = ModelCheckpoint(
    file_path,
    monitor='val_loss',
    save_best_only=True,
    mode='auto',
    verbose=1
)
callbacks_list = [checkpoint]

history = base_model.fit_generator(
    generator=train_generator_lr,
    steps_per_epoch=num_train_images // batch_size,
    epochs=epochs,
    validation_data=validation_generator_lr,
    validation_steps=800 // batch_size,
    callbacks=callbacks_list
)

common_func = common_function.CommonFunction()
common_func.plot_training(history, 'TBE-CNN (2 Branch - Epoch 40)')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 24, 24, 3)]  0                                            
__________________________________________________________________________________________________
conv1_convolution (Conv2D)      (None, 12, 12, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (MaxPooling2D)            (None

Found 424961 images belonging to 1591 classes.
Found 120531 images belonging to 1591 classes.
training: 
Epoch 1/40
13279/13280 [============================>.] - ETA: 0s - loss: 1.4956 - acc: 0.6297
Epoch 00001: val_loss improved from inf to 0.50459, saving model to tbe_cnn_ytd_2_branch.h5
13280/13280 [==============================] - 8400s 633ms/step - loss: 1.4956 - acc: 0.6297 - val_loss: 0.5046 - val_acc: 0.8825
Epoch 2/40
  825/13280 [>.............................] - ETA: 2:22:40 - loss: 1.3135 - acc: 0.6648

KeyboardInterrupt: 

In [1]:
import zipfile
with zipfile.ZipFile('ytd.zip', 'r') as zip_ref:
    zip_ref.extractall('./')